In [4]:
import graphviz as gv
# Hab das mal bei mir installiert für die Darstellung
import os

In [10]:
# https://graphviz.readthedocs.io/en/stable/manual.html

# Bildname kommt hinten in die Klammer rein
# Die dot Variable immer vor so einem Block ausführen
dot = gv.Digraph(comment='TestGraph')

dot.node('A', shape='rectangle', label='A')
dot.node('B', label='B')

dot.edge('A', 'B', arrowhead='vee')

dot.render('rectangle_arrow', view=True, format='svg')
os.remove("rectangle_arrow")

